In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
csv_path = "china_mill_data_2025_03_04_09_30_30.csv"
features = ["energy"]
input_window = 3360
output_window = 3360
batch_size = 32
epochs = 50
hidden_size = 64
learning_rate = 0.001

In [ ]:
def preprocess(csv_path, features, input_window, output_window):
    df = pd.read_csv(csv_path, parse_dates=["time"])
    df = df.sort_values("time").set_index("time")[features]

    # Store original energy for comparison
    original_energy = df['energy'].copy()

    # Handle zero values
    energy = df["energy"].copy()
    mask = energy == 0
    first_nonzero_idx = energy.ne(0).idxmax()
    mask.loc[:first_nonzero_idx] = False
    energy.loc[mask] = np.nan
    energy = energy.ffill()
    df["energy"] = energy
    df = df.ffill()

    # Plotting the original and filtered energy values
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, original_energy, label='Original Energy', alpha=0.5, linestyle='--')
    plt.plot(df.index, df['energy'], label='Filtered Energy', alpha=0.5)
    plt.title('Original vs Filtered Energy Values')
    plt.xlabel('Time')
    plt.ylabel('Energy Value')
    plt.legend()
    plt.grid(True)
    plt.show()

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(len(scaled) - input_window - output_window):
        X.append(scaled[i:i+input_window])
        y.append(scaled[i+input_window:i+input_window+output_window])
    
    X = np.array(X)
    y = np.array(y)
    print(f"✅ X shape: {X.shape}, y shape: {y.shape}")
    return X, y, scaler, df

In [ ]:
X, y, scaler, df_full = preprocess(csv_path, features, input_window, output_window)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
def build_model(input_shape, hidden_size, learning_rate):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(hidden_size, return_sequences=True))
    model.add(LSTM(hidden_size, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="mse", metrics=["mae", "mse"])
    return model

In [ ]:
model = build_model(input_shape=(input_window, 1), hidden_size=hidden_size, learning_rate=learning_rate)

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
model.save("china_mill_seq2seq_3360.h5")

In [ ]:
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
sample_input = X_test[0:1]  # Shape (1, 3360, 1)
prediction = model.predict(sample_input)  # Shape (1, 3360, 1)

In [ ]:
# Inverse transform both y_test and prediction
true_future = scaler.inverse_transform(y_test[0])
predicted_future = scaler.inverse_transform(prediction[0])

In [ ]:
# Get timestamps for plotting
start_time = df_full.index[-(output_window + 1)]
time_index = pd.date_range(start=start_time + pd.Timedelta(minutes=4), periods=output_window, freq="4min")

In [ ]:
# Plot
plt.figure(figsize=(14, 6))
plt.plot(time_index, true_future, label="Actual Energy", color="blue")
plt.plot(time_index, predicted_future, label="Predicted Energy", linestyle="--", color="orange")
plt.title("Energy Forecast - Next 3360 Points")
plt.xlabel("Time")
plt.ylabel("Energy")
plt.grid(True)
plt.legend()
plt.show()
